In [1]:
%load_ext autoreload
%autoreload 2

## Testing the Composites Discovery App with n=3 phases

In [2]:
from hashin_shtrikman_mp.core.user_input import MaterialProperty, Material, MixtureProperty, Mixture, UserInput
from hashin_shtrikman_mp.core import GeneticAlgorithm
from hashin_shtrikman_mp.core.genetic_algorithm import OptimizationParams
from hashin_shtrikman_mp.core.visualization import OptimizationResultVisualizer
from hashin_shtrikman_mp.core.match_finder import MatchFinder

/Users/carlabecker/opt/anaconda3/envs/carla_mp_api/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Define properties for each material
properties_mat_1 = [
    MaterialProperty(prop='elec_cond_300k_low_doping', upper_bound=120, lower_bound=1e-7), # upper_bound=20, lower_bound=1 
    MaterialProperty(prop='therm_cond_300k_low_doping', upper_bound=2, lower_bound=1e-7), # upper_bound=0.0001, lower_bound=1e-5 
    MaterialProperty(prop='bulk_modulus', upper_bound=500, lower_bound=50),
    MaterialProperty(prop='shear_modulus', upper_bound=500, lower_bound=80),
    MaterialProperty(prop='universal_anisotropy', upper_bound=6, lower_bound=1),
]

properties_mat_2 = [
    MaterialProperty(prop='elec_cond_300k_low_doping', upper_bound=78, lower_bound=1e-7), # upper_bound=5, lower_bound=2
    MaterialProperty(prop='therm_cond_300k_low_doping', upper_bound=2, lower_bound=1e-7), # upper_bound=0.009, lower_bound=1e-4
    MaterialProperty(prop='bulk_modulus', upper_bound=400, lower_bound=20),
    MaterialProperty(prop='shear_modulus', upper_bound=500, lower_bound=100),
    MaterialProperty(prop='universal_anisotropy', upper_bound=4.3, lower_bound=1.3),
]

properties_mat_3 = [
    MaterialProperty(prop='elec_cond_300k_low_doping', upper_bound=78, lower_bound=1e-7), # upper_bound=10, lower_bound=1
    MaterialProperty(prop='therm_cond_300k_low_doping', upper_bound=2, lower_bound=1e-7), # upper_bound=0.005, lower_bound=1e-4
    MaterialProperty(prop='bulk_modulus', upper_bound=700, lower_bound=20),
    MaterialProperty(prop='shear_modulus', upper_bound=600, lower_bound=100),
    MaterialProperty(prop='universal_anisotropy', upper_bound=2.1, lower_bound=0.9),
]

# Define properties for the mixture
properties_mixture = [
    MixtureProperty(prop='elec_cond_300k_low_doping', desired_prop=9),
    MixtureProperty(prop='therm_cond_300k_low_doping', desired_prop=0.9),
    MixtureProperty(prop='bulk_modulus', desired_prop=280),
    MixtureProperty(prop='shear_modulus', desired_prop=230),
    MixtureProperty(prop='universal_anisotropy', desired_prop=1.5),
]

# Create Material & Mixture instances
mat_1 = Material(name='mat_1', properties=properties_mat_1)
mat_2 = Material(name='mat_2', properties=properties_mat_2)
mat_3 = Material(name='mat_3', properties=properties_mat_3)
mixture = Mixture(name='mixture', properties=properties_mixture)
aggregate = [mat_1, mat_2, mat_3, mixture]

# Initialize UserInput instance with materials and mixtures
user_input= UserInput(materials=[mat_1, mat_2, mat_3], mixtures=[mixture])
print("User Input: ", user_input)

# Initialize dictionaries to store the overall upper and lower bounds for each property
overall_bounds = {}

# Iterate over materials
for entity in aggregate:
    # Skip the mixture as it doesn't have upper and lower bounds
    if isinstance(entity, Material):
        for property in entity.properties:
            prop_name = property.prop

            # Initialize the overall bounds if they are not already present for the property
            if prop_name not in overall_bounds:
                overall_bounds[prop_name] = {'upper_bound': property.upper_bound, 'lower_bound': property.lower_bound}
            else:
                # Update overall upper and lower bounds by comparing with existing values
                overall_bounds[prop_name]['upper_bound'] = max(overall_bounds[prop_name]['upper_bound'], property.upper_bound)
                overall_bounds[prop_name]['lower_bound'] = min(overall_bounds[prop_name]['lower_bound'], property.lower_bound)

# Print the overall bounds for each property
print("Overall Upper & Lower Bounds:")
for prop, bounds in overall_bounds.items():
    print(f"Property: {prop}, Upper Bound: {bounds['upper_bound']}, Lower Bound: {bounds['lower_bound']}")

# Step 1: Create the consolidated_dict
overall_bounds_dict = {}
for prop, bounds in overall_bounds.items():
    overall_bounds_dict[prop] = {
        'upper_bound': bounds['upper_bound'],
        'lower_bound': bounds['lower_bound']
    }

print(overall_bounds_dict)

User Input:  {'mat_1': {'elec_cond_300k_low_doping': {'upper_bound': 120.0, 'lower_bound': 1e-07}, 'therm_cond_300k_low_doping': {'upper_bound': 2.0, 'lower_bound': 1e-07}, 'bulk_modulus': {'upper_bound': 500.0, 'lower_bound': 50.0}, 'shear_modulus': {'upper_bound': 500.0, 'lower_bound': 80.0}, 'universal_anisotropy': {'upper_bound': 6.0, 'lower_bound': 1.0}}, 'mat_2': {'elec_cond_300k_low_doping': {'upper_bound': 78.0, 'lower_bound': 1e-07}, 'therm_cond_300k_low_doping': {'upper_bound': 2.0, 'lower_bound': 1e-07}, 'bulk_modulus': {'upper_bound': 400.0, 'lower_bound': 20.0}, 'shear_modulus': {'upper_bound': 500.0, 'lower_bound': 100.0}, 'universal_anisotropy': {'upper_bound': 4.3, 'lower_bound': 1.3}}, 'mat_3': {'elec_cond_300k_low_doping': {'upper_bound': 78.0, 'lower_bound': 1e-07}, 'therm_cond_300k_low_doping': {'upper_bound': 2.0, 'lower_bound': 1e-07}, 'bulk_modulus': {'upper_bound': 700.0, 'lower_bound': 20.0}, 'shear_modulus': {'upper_bound': 600.0, 'lower_bound': 100.0}, 'unive

In [4]:
# Initialize optimization parameters and genetic algorithm
optimization_parameters = OptimizationParams.from_user_input(user_input)
ga = GeneticAlgorithm()

print("Property categories: ", optimization_parameters.property_categories)
print("Property docs: ", optimization_parameters.property_docs)

print("Lower Bounds:", optimization_parameters.lower_bounds)
print("Upper Bounds:", optimization_parameters.upper_bounds)
print("Desired Properties:", optimization_parameters.desired_props)

print("Number of Materials:", optimization_parameters.num_materials)
print("Number of Properties:", optimization_parameters.num_properties)

2025-01-23 12:04:25,842 - hashin_shtrikman_mp.log.custom_logger - INFO - Loading property categories from /Users/carlabecker/Library/Mobile Documents/com~apple~CloudDocs/Carla's Desktop/UC Berkeley/Research/Materials Project/hashin_shtrikman_mp/src/hashin_shtrikman_mp/io/inputs/data/mp_property_docs.yaml.
2025-01-23 12:04:25,849 - hashin_shtrikman_mp.log.custom_logger - INFO - property_categories = ['carrier-transport', 'elastic']
2025-01-23 12:04:25,850 - hashin_shtrikman_mp.log.custom_logger - INFO - mixture_props = {'elec_cond_300k_low_doping': {'desired_prop': 9.0}, 'therm_cond_300k_low_doping': {'desired_prop': 0.9}, 'bulk_modulus': {'desired_prop': 280.0}, 'shear_modulus': {'desired_prop': 230.0}, 'universal_anisotropy': {'desired_prop': 1.5}}


Property categories:  ['carrier-transport', 'elastic']
Property docs:  {'carrier-transport': {'elec_cond_300k_low_doping': None, 'therm_cond_300k_low_doping': None}, 'dielectric': {'e_electronic': None, 'e_ionic': None, 'e_total': None, 'n': None}, 'elastic': {'bulk_modulus': 'vrh', 'shear_modulus': 'vrh', 'universal_anisotropy': None}, 'magnetic': {'total_magnetization': None, 'total_magnetization_normalized_vol': None}, 'piezoelectric': {'e_ij_max': None}}
Lower Bounds: {'mat_1': {'carrier-transport': [1e-07, 1e-07], 'elastic': [50.0, 80.0, 1.0]}, 'mat_2': {'carrier-transport': [1e-07, 1e-07], 'elastic': [20.0, 100.0, 1.3]}, 'mat_3': {'carrier-transport': [1e-07, 1e-07], 'elastic': [20.0, 100.0, 0.9]}, 'volume-fractions': [0.01, 0.01, 0.01]}
Upper Bounds: {'mat_1': {'carrier-transport': [120.0, 2.0], 'elastic': [500.0, 500.0, 6.0]}, 'mat_2': {'carrier-transport': [78.0, 2.0], 'elastic': [400.0, 500.0, 4.3]}, 'mat_3': {'carrier-transport': [78.0, 2.0], 'elastic': [700.0, 600.0, 2.1]},

In [5]:
# Run the optimization to obtain n=3 materials and their volume fractions that achieve the desired composite material properties
ga_result = ga.run(user_input, gen_counter=True)

2025-01-23 12:04:25,903 - hashin_shtrikman_mp.log.custom_logger - INFO - Loading property categories from /Users/carlabecker/Library/Mobile Documents/com~apple~CloudDocs/Carla's Desktop/UC Berkeley/Research/Materials Project/hashin_shtrikman_mp/src/hashin_shtrikman_mp/io/inputs/data/mp_property_docs.yaml.
2025-01-23 12:04:25,908 - hashin_shtrikman_mp.log.custom_logger - INFO - property_categories = ['carrier-transport', 'elastic']
2025-01-23 12:04:25,910 - hashin_shtrikman_mp.log.custom_logger - INFO - mixture_props = {'elec_cond_300k_low_doping': {'desired_prop': 9.0}, 'therm_cond_300k_low_doping': {'desired_prop': 0.9}, 'bulk_modulus': {'desired_prop': 280.0}, 'shear_modulus': {'desired_prop': 230.0}, 'universal_anisotropy': {'desired_prop': 1.5}}


Generation 0 of 100
Generation 1 of 100
Generation 2 of 100
Generation 3 of 100
Generation 4 of 100
Generation 5 of 100
Generation 6 of 100
Generation 7 of 100
Generation 8 of 100
Generation 9 of 100
Generation 10 of 100
Generation 11 of 100
Generation 12 of 100
Generation 13 of 100
Generation 14 of 100
Generation 15 of 100
Generation 16 of 100
Generation 17 of 100
Generation 18 of 100
Generation 19 of 100
Generation 20 of 100
Generation 21 of 100
Generation 22 of 100
Generation 23 of 100
Generation 24 of 100
Generation 25 of 100
Generation 26 of 100
Generation 27 of 100
Generation 28 of 100
Generation 29 of 100
Generation 30 of 100
Generation 31 of 100
Generation 32 of 100
Generation 33 of 100
Generation 34 of 100
Generation 35 of 100
Generation 36 of 100
Generation 37 of 100
Generation 38 of 100
Generation 39 of 100
Generation 40 of 100
Generation 41 of 100
Generation 42 of 100
Generation 43 of 100
Generation 44 of 100
Generation 45 of 100
Generation 46 of 100
Generation 47 of 100
Ge

In [6]:
# Create an instance of the Visualizer class using the genetic algorithm result
visualizer = OptimizationResultVisualizer(ga_result)

In [7]:
# Print the optimization results as a table
visualizer.print_table_of_best_designs(rows=10)

In [8]:
# Plot the genetic algorithm convergence plot
visualizer.plot_optimization_results()

In [9]:
# Plot the contributions to the cost function for the best performer
visualizer.plot_cost_func_contribs()

Cost of best design: 0.05562515601855033


In [10]:
# Create an instance of the MatchFinder class using the genetic algorithm result
match_finder = MatchFinder(ga_result)

In [11]:
# Get material matches
matches_dict = match_finder.get_material_matches(overall_bounds_dict)
print(f'Material Matches: {matches_dict}')

overall_bounds_dict: {'elec_cond_300k_low_doping': {'upper_bound': 120.0, 'lower_bound': 1e-07}, 'therm_cond_300k_low_doping': {'upper_bound': 2.0, 'lower_bound': 1e-07}, 'bulk_modulus': {'upper_bound': 700.0, 'lower_bound': 20.0}, 'shear_modulus': {'upper_bound': 600.0, 'lower_bound': 80.0}, 'universal_anisotropy': {'upper_bound': 6.0, 'lower_bound': 0.9}}


Retrieving SummaryDoc documents: 100%|██████████| 210/210 [00:00<00:00, 842874.49it/s]


Material Matches: {'mat1': [{'mp-684591': {'elec_cond': 87.8933, 'therm_cond': 0.000354247, 'bulk_modulus': 141.633, 'shear_modulus': 83.335, 'universal_anisotropy': 1.2690000000000001}}, {'mp-752826': {'elec_cond': 41.0475, 'therm_cond': 0.000111994, 'bulk_modulus': 166.16, 'shear_modulus': 81.144, 'universal_anisotropy': 1.271}}, {'mp-3536': {'elec_cond': 77.0117, 'therm_cond': 0.000316911, 'bulk_modulus': 182.861, 'shear_modulus': 98.235, 'universal_anisotropy': 1.062}}, {'mp-4391': {'elec_cond': 110.485, 'therm_cond': 0.000659704, 'bulk_modulus': 199.288, 'shear_modulus': 108.878, 'universal_anisotropy': 1.601}}, {'mp-753459': {'elec_cond': 114.842, 'therm_cond': 0.000524039, 'bulk_modulus': 212.133, 'shear_modulus': 90.785, 'universal_anisotropy': 3.167}}, {'mp-2657': {'elec_cond': 117.021, 'therm_cond': 0.000614014, 'bulk_modulus': 208.538, 'shear_modulus': 107.377, 'universal_anisotropy': 1.629}}, {'mp-755759': {'elec_cond': 87.3874, 'therm_cond': 0.000486809, 'bulk_modulus': 21